In [2]:
! pip install -qU memorizz

In [3]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [4]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [6]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

In [7]:
set_env_securely("VOYAGE_API_KEY", "Enter your VOYAGE_API_KEY: ")

In [8]:
from memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a mongodb config with voyageai embeddings
mongodb_config = MongoDBConfig(
    uri=os.environ["MONGODB_URI"],
    db_name="testing_memorizz",
    embedding_provider="voyageai",
    embedding_config={
        "embedding_type": "contextualized",
        "model": "voyage-context-3",
        "output_dimension": 256,
    }
)

# Create a memory provider
memory_provider = MongoDBProvider(mongodb_config)

/Users/richmondalake/miniconda3/envs/agent_memory_course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector search index 'vector_index' already exists for collection personas.
Vector search index 'vector_index' already exists for collection toolbox.
Vector search index 'vector_index' already exists for collection short_term_memory.
Vector search index 'vector_index' already exists for collection long_term_memory.
Vector search index 'vector_index' already exists for collection conversation_memory.
Vector search index 'vector_index' already exists for collection workflow_memory.
Vector search index 'vector_index' already exists for collection agents.
Vector search index 'vector_index' already exists for collection shared_memory.
Vector search index 'vector_index' already exists for collection summaries.


In [12]:
from memorizz.long_term_memory.semantic.persona import Persona, RoleType

# Create a persona for our agent
persona = Persona(
    name="Alex",
    role=RoleType.ASSISTANT,
    goals="Help users learn and provide thoughtful assistance",
    background="An AI assistant focused on being helpful, educational, and adaptive"
)

In [13]:
from memorizz.memagent import MemAgent
from memorizz.llms.openai import OpenAI

# Create an agent with the persona
agent = MemAgent(
    model=OpenAI(model="gpt-4"),
    persona=persona,
    instruction="You are a helpful learning assistant that adapts based on interactions",
    application_mode="assistant",
    memory_provider=memory_provider
)

In [14]:
agent.save()

MemAgent(agent_id=689ca9dd8fc3ee839edbfc35, memory_provider=<memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x118d2d010>)

In [15]:
print(f"Created agent: {agent.agent_id}")
print(f"Initial persona goals: {agent.persona.goals}")
print(f"Initial persona background: {agent.persona.background}")

Created agent: 689ca9dd8fc3ee839edbfc35
Initial persona goals: Assist users by offering timely and personalized support. Help users learn and provide thoughtful assistance
Initial persona background: An assistant agent crafted to manage schedules, answer queries, and help with daily tasks. An AI assistant focused on being helpful, educational, and adaptive


In [16]:
# Simulate some interactions to create memory components
interactions = [
    "Can you help me understand machine learning?",
    "I'm struggling with Python programming concepts",
    "What are the best practices for data visualization?", 
    "How do I improve my debugging skills?",
    "Can you explain neural networks in simple terms?",
    "I need help with statistical analysis",
    "What's the difference between supervised and unsupervised learning?",
    "How do I handle missing data in datasets?",
    "Can you recommend learning resources for data science?",
    "I'm working on a classification problem and need guidance"
]

In [17]:
import time

# Process interactions to build up memory
for i, query in enumerate(interactions):
    print(f"Processing interaction {i+1}: {query[:50]}...")
    response = agent.run(query)
    print(f"  Response: {response[:100]}...")
    
    # Add a small delay to create time separation
    time.sleep(1)

print(f"\nAgent now has {len(agent.memory_ids)} memory sessions")

Processing interaction 1: Can you help me understand machine learning?...
  Response: Sure, I'd be happy to help you understand machine learning. At its core, machine learning is a metho...
Processing interaction 2: I'm struggling with Python programming concepts...
  Response: Sure, I'd be happy to help you with Python programming concepts. Python is a widely-used, interprete...
Processing interaction 3: What are the best practices for data visualization...
  Response: Sure, here are some broadly recognized best practices for data visualization:

1. **Choose the Right...
Processing interaction 4: How do I improve my debugging skills?...
  Response: Debugging is a crucial skill for any programmer and improving this skill can greatly enhance your ef...
Processing interaction 5: Can you explain neural networks in simple terms?...
  Response: Sure, I'd be happy to explain neural networks in simple terms.

A neural network is a type of machin...
Processing interaction 6: I need help with s

In [18]:
# Generate summaries from the accumulated memories
# 💡 Tip: This can be placed in background task that's triggered by a cron job
# 💡 Tip: Or it can be provided to the agent as a tool to be used at it's discretion
summary_ids = agent.generate_summaries(
    days_back=1,  # Look back 1 day (covers all our interactions)
    max_memories_per_summary=5  # Smaller chunks for demo
)

In [19]:
print(f"Generated {len(summary_ids)} summaries:")
for i, summary_id in enumerate(summary_ids):
    print(f"  Summary {i+1}: {summary_id}")

Generated 4 summaries:
  Summary 1: 689cab328fc3ee839edbfc55
  Summary 2: 689cab438fc3ee839edbfc56
  Summary 3: 689cab508fc3ee839edbfc57
  Summary 4: 689cab5c8fc3ee839edbfc58


In [22]:
from datetime import datetime
from memorizz import MemoryType

# Show what the summaries look like
print("\n--- Examining Summaries ---")
if summary_ids:
    # Get the first summary to show its content
    summary = memory_provider.retrieve_by_id(summary_ids[0], MemoryType.SUMMARIES)
    if summary:
        print(f"Sample summary content:")
        
        # Debug: Check what format the timestamps are in
        print(f"  Raw period_start: {summary['period_start']} (type: {type(summary['period_start'])})")
        print(f"  Raw period_end: {summary['period_end']} (type: {type(summary['period_end'])})")
        
        # Try to parse timestamps (handle both numeric strings and ISO format)
        try:
            # If they're numeric strings, convert to float first
            start_time = float(summary['period_start'])
            end_time = float(summary['period_end'])
            start_dt = datetime.fromtimestamp(start_time)
            end_dt = datetime.fromtimestamp(end_time)
            print(f"  Period: {start_dt} to {end_dt}")
        except ValueError:
            # If they're ISO format strings, parse directly
            try:
                start_dt = datetime.fromisoformat(summary['period_start'])
                end_dt = datetime.fromisoformat(summary['period_end'])
                print(f"  Period: {start_dt} to {end_dt}")
            except ValueError:
                # If neither works, just show the raw values
                print(f"  Period: {summary['period_start']} to {summary['period_end']} (unknown format)")
        
        print(f"  Memory units: {summary['memory_units_count']}")
        print(f"  Content: {summary['summary_content'][:400]}...")


--- Examining Summaries ---
Sample summary content:
  Raw period_start: 1755097574.076263 (type: <class 'float'>)
  Raw period_end: 1755097616.510677 (type: <class 'float'>)
  Period: 2025-08-13 16:06:14.076263 to 2025-08-13 16:06:56.510677
  Memory units: 5
  Content: The individual demonstrated a desire to understand complex topics, starting with machine learning. They learned about different types of machine learning, such as supervised learning, unsupervised learning, and reinforcement learning. They also understood how machine learning algorithms are beneficial in scenarios where defining deterministic rules is challenging but ample data is available.

Foll...


In [23]:
# Update persona based on summaries
# 💡 Tip: This can be placed in background task that's triggered by a cron job
# 💡 Tip: Or it can be provided to the agent as a tool to be used at it's discretion
persona_updated = agent.update_persona_from_summaries(
    max_summaries=3,  # Use up to 3 most recent summaries
    save=True         # Save the updated persona
)

Storing persona: Alex in the memory provider, in the personas collection


In [24]:
if persona_updated:
    print("✅ Persona successfully updated!")
    print(f"Updated goals: {agent.persona.goals}")
    print(f"Updated background: {agent.persona.background}")
else:
    print("❌ Persona update failed")



✅ Persona successfully updated!
Updated goals: Continue to assist users by offering in-depth and personalized support. Guide users in complex areas such as data science, machine learning, and data visualization. Promote continuous learning, simplify complex concepts, and provide thoughtful assistance with refined expertise and accuracy.
Updated background: An evolved AI assistant with an advanced understanding of technical areas like data science, machine learning and statistical analysis. Skilled in explaining intricate concepts in manageable terms, offering detailed learning resources, crucial problem-solving strategies and coaching in cognitively challenging tasks. Proven expertise in patiently fostering user understanding and boosting their confidence in managing complex subjects.


In [25]:
# Get all summaries for this agent
if agent.memory_ids:
    memory_id = agent.memory_ids[0]
    all_summaries = memory_provider.get_summaries_by_memory_id(memory_id, limit=10)
    print(f"Found {len(all_summaries)} summaries for memory_id {memory_id}")
    
    # Get summaries within a time range
    current_time = time.time()
    start_time = current_time - (24 * 60 * 60)  # Last 24 hours
    time_range_summaries = memory_provider.get_summaries_by_time_range(
        memory_id, start_time, current_time
    )
    print(f"Found {len(time_range_summaries)} summaries in the last 24 hours")


Found 4 summaries for memory_id 689ca9e38fc3ee839edbfc36
Found 4 summaries in the last 24 hours


In [26]:
agent.save()
print(f"\nAgent saved with ID: {agent.agent_id}")


Agent saved with ID: 689ca9dd8fc3ee839edbfc35
